In [10]:
from sklearn.externals import joblib

print("Starting evaluation")
print("Loading model")
model = joblib.load("model.pkl")
print("Model loaded")

Starting evaluation
Loading model
Model loaded


Get evaluation data

In [15]:
import pandas as pd

test_df = pd.read_csv("test_data.csv")

X = test_df.iloc[:,1:]
y = test_df.iloc[:,0]
#Temp step, this should be integrated in a seperate 'pre-processing' program
#Normalize test data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1), copy=True)
X = scaler.fit_transform(X)

In [16]:
y_pred = model.predict(X)

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
cm = confusion_matrix(y, y_pred)
accuracy = accuracy_score(y, y_pred)

print("Test results\n-------------------------------------------------------------\n")
print("Accuracy: {0}%\n".format(accuracy))
print("Confusion matrix:")
print(cm)

Test results
-------------------------------------------------------------

Accuracy: 0.878095238095238%

Confusion matrix:
[[799   0   3   5   6   5  11   2  13   8]
 [  0 911   4   6   0  12   4   2  10   1]
 [ 13   5 722  15  10   2   7   9  12   6]
 [  9  17  42 704   1  47  10  11  19  22]
 [  1   8   5   0 701   2  11   2  11  70]
 [ 16  16  11  18  20 618  16   2  10  20]
 [  9   5   8   0  14  11 772   0  12   0]
 [  7   6  19   1   8   5   0 798   3  37]
 [  3  36  28   8  15  22   6   3 640  41]
 [  9   5  10   5  41  11   1  38   9 711]]


In [24]:
if accuracy > 85:
    print("\nModel passed the test.")
    returnvalue = 0
else:
    print("Model did not pass the test.")    
    returnvalue = 1
import sys
sys.exit(returnvalue)

Model did not pass the test.


SystemExit: 1

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
